# Modulus for hypertrees

In [25]:
from modulus_tools.basic_algorithm import matrix_modulus, modulus
import itertools
from itertools import combinations, chain
import numpy as np
from IPython.display import Image, display

## Hypergraphs

A **hypergraph** is an ordered pair $H = (V, E)$ where:

- $V$ is a finite set whose elements are called **vertices**, and  
- $E$ is a multiset (or set) of nonempty subsets of $V$, whose elements are called **hyperedges**

<div align="center">

```{figure} images/si-hyp.png
:width: 400px
:class: no-caption-number

*A hypergraph with 5 vertices and 4 hyperedges.*
```

## Hypertrees

Let $H = (V, E)$ be a hypergraph. A subset \( F \subseteq E \) is a **hyperforest** if it is possible to select two distinct vertices from each hyperedge \( e \in F \) such that the chosen pairs, when regarded as graph edges on $V$ , form a forest.  
If that representing forest can be made into a spanning tree, then \( F \) is a **hypertree**.

<div align="center">

```{figure} images/si-tree.png
:width: 400px
:class: no-caption-number

*A hypertree with a spanning tree.*
```

## Equivalent definition of hypertrees 

For a non-empty set $X \subseteq V$ and a subset 
$F \subseteq E$, the set $F[X]$ represents the collection of hyperedges in $F$ that 
are entirely contained in $X$. The hypergraph $(X, E[X])$ is called the *subhypergraph 
induced by the vertex subset* $X$ and is denoted by $H[X]$.

A set $F \subseteq E$ is a **hyperforest** if $|F[X]| \le |X| - 1$ for every 
non-empty subset $X \subseteq V$. A hyperforest $F$ is a **hypertree** of $H$ 
if $|F| = |V| - 1$.

## Usage matrix
Next, we are going to find all indicator vectors of all hypertrees.

In [26]:
# Helper function to get the powerset of a list
def powerset(iterable):
    "powerset([1,2,3]) --> [(), (1,), (2,), (3,), (1,2), (1,3), (2,3), (1,2,3)]"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))
# Output the hypertree family
def trees(V,E):
    m = len(E)
    n = len(V)
    dN = []
    for J in combinations(range(len(E)), len(V)-1):
        Jgood = True
        for X in list(powerset(V))[1:]:
            # Check the subsets of E indexed by J
            if not len([F for F in [E[i] for i in J] if set(F).issubset(X)]) <= len(X) - 1:
                Jgood = False
                break
        if Jgood:
            row = np.zeros(len(E))
            for i in J:
                row[i % m] = row[i % m]+1
            dN.append(row)
    N = np.unique(dN, axis=0) 
    return(N)

In [27]:
# Define the hyperedge set E of the hypergraph H = (V, E)
# Each hyperedge is a subset of vertices in V\# Define the number of vertices
n = 6
V = range(n)
E = [[0,1,2],[0,1,3],[1,2,3],[3,5],[3,4],[4,5],[0,5,2]]
T = trees(V, E)
print('Usage matrix:')
print(T)

Usage matrix:
[[0. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1.]
 [1. 0. 1. 1. 0. 1. 1.]
 [1. 0. 1. 1. 1. 0. 1.]
 [1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 0. 1.]
 [1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 0. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1. 0.]
 [1. 1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 0. 0.]]


## Hypertree modulus

we compute 2-modulus and 1-modulus of the hypertree family

In [28]:
# Compute the 2-modulus (p = 2)
mod2, rho2, _ = matrix_modulus(T, 2)
print("=== 2-Modulus Results ===")
print(f"Modulus value (mod₂): {mod2:.4f}")
print(f"Optimal density η*   : {np.round(rho2 / mod2, 2)}\n")

# Compute the 1-modulus (p = 1)
mod1, rho1, _ = matrix_modulus(T, 1)
print("=== 1-Modulus Results ===")
print(f"Modulus value (mod₁): {mod1:.4f}")


=== 2-Modulus Results ===
Modulus value (mod₂): 0.2791
Optimal density η*   : [0.75 0.75 0.75 0.67 0.67 0.67 0.75]

=== 1-Modulus Results ===
Modulus value (mod₁): 1.3333


## Strength and fractional arboricity

Let $\mathcal{P} = \{V_1, \ldots, V_k\}$ be a partition of the vertex set $V$, and 
let $F \subseteq E$. We define $\delta(\mathcal{P})$ as the set of hyperedges that intersect at least two parts of the partition $\mathcal{P}$. The notation 
$|\mathcal{P}|$ represents the number of parts in the partition $\mathcal{P}$.  

Following the concepts of strength and fractional arboricity in graphs, we define 
the *strength* of a hypergraph $H$ as  

$$
S(H) := \min \left\{ \frac{|\delta(\mathcal{P})|}{|\mathcal{P}| - 1} :
\mathcal{P} \text{ is a partition of } V, \ |\mathcal{P}| \ge 2 \right\},
$$

and define the *fractional arboricity* of $H$ as  

$$
D(H) := \max \left\{ \frac{|E[X]|}{|X| - 1} : X \subseteq V, \ |X| \ge 2 \right\}.
$$

In [29]:
## Compute the strength of a hypergraph
## All partition of a set
def partitions(set_):
    if len(set_) == 1:
        yield [list(set_)]
        return
    first_element = set_[0]
    for smaller_partition in partitions(set_[1:]):
        # Insert first element into each part of the partition
        for i in range(len(smaller_partition)):
            new_partition = smaller_partition[:i] + [[first_element] + smaller_partition[i]] + smaller_partition[i+1:]
            yield new_partition
        # Or create a new part with the first element <class 'generator'>
        yield [[first_element]] + smaller_partition

def strength(V,E, sigma = None):
    if sigma is None:
        sigma = [1]*len(E)
    strength = sum(sigma)/(len(V)-1)
    for p in list(partitions(V)):
        if len(p)>1:
            if sum([sigma[i] for i in range(len(E)) if not any(set(E[i]).issubset(b) for b in p)])/(len(p)-1) < strength:
               strength = sum([sigma[i] for i in range(len(E)) if not any(set(E[i]).issubset(b) for b in p)]) /(len(p)-1)
    return strength
def fractional_arboricity(V, E, sigma=None):
    """
    Compute the weighted fractional arboricity D_sigma(H) of a hypergraph H = (V, E):
        D_sigma(H) = max_{X ⊆ V, |X| ≥ 2} σ(E[X]) / (|X| - 1)
    where σ(E[X]) = sum of weights of edges fully contained in X.

    Parameters
    ----------
    V : iterable
        Vertex set (e.g., range(n) or list of vertices)
    E : list of iterables
        Hyperedges; each hyperedge is an iterable of vertices in V
    sigma : list or array-like, optional
        Weights corresponding to each hyperedge; defaults to all 1's

    Returns
    -------
    float
        Weighted fractional arboricity D_sigma(H)
    """
    V = list(V)
    if len(V) < 2:
        raise ValueError("D_sigma(H) is undefined for |V| < 2.")

    if sigma is None:
        sigma = [1.0] * len(E)
    if len(sigma) != len(E):
        raise ValueError("sigma must have the same length as E.")

    E_sets = [set(e) for e in E]
    D = 0.0

    # Iterate over all subsets X with |X| ≥ 2
    for k in range(2, len(V) + 1):
        for X in combinations(V, k):
            Xset = set(X)
            # Sum of weights of edges contained in X
            weighted_sum = sum(sigma[i] for i, e in enumerate(E_sets) if e.issubset(Xset))
            D = max(D, weighted_sum / (k - 1))
    return D

In [30]:
#Example 
n = 6
V = range(n)
E = [[0,1,2],[0,1,3],[1,2,3],[3,5],[3,4],[4,5],[0,5,2]]
print('Strength = ',np.round(strength(V,E),2))
print('Fractional_arboricity =', np.round(fractional_arboricity(V, E),2))

Strength =  1.33
Fractional_arboricity = 1.5


## Hypertree 1-modulus and strength
Let $H$ be a partition-connected hypergraph. Then,

$$
\operatorname{Mod}_1(\Gamma(H)) = S(H).
$$



In [31]:
#Example: 
n = 6
V = range(n)
E = [[0,1,2],[0,1,3],[1,2,3],[3,5],[3,4],[4,5],[0,5,2]]

mod1, rho1, _ = matrix_modulus(T, 1)
print('1-modulus: mod = ',np.round(mod1,2))
print('Strength = ',np.round(strength(V,E),2))

1-modulus: mod =  1.33
Strength =  1.33


## Hypertree 2-modulus encodes strength and fractional arboricity
Let $H$ be a partition-connected hypergraph with weights $\sigma \in \mathbb{R}^E_{>0}$.

For 2-modulus, let $\eta^*$ be the optimal density of $\operatorname{Mod}_{2}(\widehat{\Gamma})$, we have


$$
\frac{1}{(\sigma^{-1}\eta)^*_{max}}= S_{\sigma}(H), \qquad \frac{1}{(\sigma^{-1}\eta)^*_{min}}= D_{\sigma}(H).
$$

In [32]:
#Example: 
n = 4
V = range(n)
E = [[0,1,2],[0,1,2],[0,3]]
T = trees(V, E)
mod2, rho2, _ = matrix_modulus(T, 2)
print('2-modulus: mod = ',np.round(mod2,2))
print('2-modulus: eta^* = ',np.round(rho2/mod2,2))
print('Strength = ',np.round(strength(V,E),2))
print('Fractional_arboricity =', np.round(fractional_arboricity(V, E),2))

2-modulus: mod =  0.33
2-modulus: eta^* =  [1. 1. 1.]
Strength =  1.0
Fractional_arboricity = 1.0


In [33]:
#Example 2
n = 6
V = range(n)
E = [[0,1,2],[0,1,3],[1,2,3],[3,5],[3,4],[4,5],[0,5,2]]
T = trees(V, E)
mod2, rho2, _ = matrix_modulus(T, 2)
print('2-modulus: mod = ',np.round(mod2,2))
print('2-modulus: eta^* = ',np.round(rho2/mod2,2))
print('Strength = ',np.round(strength(V,E),2))
print('Fractional_arboricity =', np.round(fractional_arboricity(V, E),2))


2-modulus: mod =  0.28
2-modulus: eta^* =  [0.75 0.75 0.75 0.67 0.67 0.67 0.75]
Strength =  1.33
Fractional_arboricity = 1.5


## Multitree hypergraph

A natural question arises: *what happens when $H$ is not partition-connected?*  
Does modulus still reveal a hierarchical structure? The answer is **yes**.

To explain this, we introduce the family of **multitrees** $\Omega = \Omega(H)$ as follows:  Each multitree is a vector $x \in \mathbb{R}^E$ where $x(e)$ counts how many copies of a hyperedge $e$ are included such that their union forms a hypertree on $V$.

**Example**  
Consider a hypergraph $H = (V, E)$ with  $V = \{v_1, v_2, v_3\}$,  $E = \{e_1, e_2\}$, and  $e_1 = e_2 = \{v_1, v_2, v_3\}$.  

<div align="center">

```{figure} images/si-mul.png
:width: 300px
:class: no-caption-number

*$\Omega(H)=\{(2, 1)\}$*
```

<div align="center">

```{figure} images/hyper1.png
:width: 250px
:class: no-caption-number

*$\Omega(H)=\{(1, 1), (2, 0), (0, 2)\}$*
```

## Usage matrix
Next, we are going to find the usage matrix of the multitree family

In [34]:
# Output the multitree family
def multitrees(V,E):
    m = len(E)
    n = len(V)
    dN = []
    cE = E * (n-1)
    for J in combinations(range(len(cE)), len(V)-1):
        Jgood = True
        for X in list(powerset(V))[1:]:
            # Check the subsets of E indexed by J
            if not len([F for F in [cE[i] for i in J] if set(F).issubset(X)]) <= len(X) - 1:
                Jgood = False
                break
        if Jgood:
            row = np.zeros(len(E))
            for i in J:
                row[i % m] = row[i % m]+1
            dN.append(row)
    N = np.unique(dN, axis=0)
    return(N)

In [35]:
#Example: 
n = 5
V = range(n)
E = [[0,1,2,3],[0,1],[2,3],[3,4],[3,4]]
N = multitrees(V,E)
print('Usage matrix of the multitree family:')
print(N)

Usage matrix of the multitree family:
[[1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0.]
 [2. 0. 1. 0. 1.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 1.]
 [2. 1. 0. 1. 0.]
 [3. 0. 0. 0. 1.]
 [3. 0. 0. 1. 0.]]


In [36]:
# Compute the 2-modulus (p = 2)
mod2, rho2, _ = matrix_modulus(N, 2)
print("=== 2-Modulus Results ===")
print(f"Modulus value (mod₂): {mod2:.4f}")
print(f"Optimal density η*   : {np.round(rho2 / mod2, 2)}\n")

# Compute the 1-modulus (p = 1)
mod1, rho1, _ = matrix_modulus(N, 1)
print("=== 1-Modulus Results ===")
print(f"Modulus value (mod₁): {mod1:.4f}")
print('Strength = ',np.round(strength(V,E),2))
print('Fractional_arboricity =', np.round(fractional_arboricity(V, E),2))

=== 2-Modulus Results ===
Modulus value (mod₂): 0.2857
Optimal density η*   : [1.  1.  1.  0.5 0.5]

=== 1-Modulus Results ===
Modulus value (mod₁): 1.0000
Strength =  1.0
Fractional_arboricity = 2.0


## Multitrees modulus properties
We have

$$
\operatorname{Mod}_1(\Omega(H)) = S(H).
$$

Let $\eta^*$ be the optimal density of $\operatorname{Mod}_{2}(\widehat{\Omega})$, we have

$$
S(H) = \frac{1}{\eta^*_{\max}}, 
\qquad 
D(H) = \frac{1}{\eta^*_{\min}}.
$$

<p align="center">
A hypergraph $H$ for which $S(H) = D(H)$ is called a 
**homogeneous** hypergraph.
</p>


If $H$ is partition-connected, then $\operatorname{Mod}_{2}(\widehat{\Omega})$ and $\operatorname{Mod}_{2}(\widehat{\Gamma})$ have the same optimal density.


## Decomposition process
We also propose a **deflation process** for $H$ that identifies a hierarchical structure of hypergraphs in which each subhypergraph of the deflation preserves $\eta^*$, and its strength and fractional arboricity are likewise encoded in $\eta^*$.




- Red hyperedges: $\eta^* = \tfrac{3}{5}$
- Green hyperedges: $\eta^* = 1$
- Blue hyperedges: $\eta^* = \tfrac{3}{2}$



<div align="center">

```{figure} images/de-part.png
:width: 400px
:class: no-caption-number
```


$$
\downarrow
$$


<div align="center">

```{figure} images/de-s1-part.png
:width: 400px
:class: no-caption-number
```

$$
\downarrow
$$


<div align="center">

```{figure} images/de-s2.png
:width: 250px
:class: no-caption-number
```